<a href="https://colab.research.google.com/github/dariru3/py-comment_pdf/blob/main/pdf_highlighter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pymupdf
!pip install frontend

In [ ]:
def comment_pdf(input_file:str, list_filename_csv:str, output_file:str, pages:list=None):
    import fitz, csv
    comment_title = "Python Highlighter"
    colors = {"red": [0.7, 0.35, 0.5], "green": [0.35, 0.7, 0.5], "blue": [0.35, 0.5, 0.7]}
    with open(list_filename_csv, 'r') as csv_data:
        csv_reader = csv.reader(csv_data)
        header = next(csv_reader) # skips the first row
        search_list = [row for row in csv_reader]

    pdfIn = fitz.open(input_file)
    
    # create matches dictionary for output summary
    matches_record = {search[0]: 0 for search in search_list}
    
    # Iterate throughout the document pages
    for pg,page in enumerate(pdfIn):
        pageID = pg+1
        # If required for specific pages
        if pages and pageID not in pages:
            continue

        # Use the search_for function to find the text
        for search_settings in search_list:
            word, comment, color = search_settings
            matched_values = page.search_for(word,hit_max=20)
            if matched_values:
                matches_record[word] += len(matched_values)
                #Loop through the matched values; item will contain the coordinates of the found text
                for item in matched_values:
                    # Highlight found text
                    annot = page.add_highlight_annot(item)
                    if color:
                        annot.set_colors(stroke=colors[color])
                    # Add comment to the found match
                    info = annot.info
                    info["title"] = comment_title
                    info["content"] = comment
                    annot.set_info(info)
                    annot.update()
    
    #Save to output file
    pdfIn.save(output_file,garbage=3,deflate=True)
    pdfIn.close()

    #Process Summary
    summary = {
        "Input File": input_file
        , "Output File": output_file
        , "Comment Title": comment_title
        , "Matching Instances": "\n" + "\n".join("{}: {}".format(word, count) for word, count in matches_record.items())
    }

    # Export Process Summary
    with open('summary.txt', 'w') as summary_txt:
        summary_txt.write("\n".join("{}: {}".format(i, j) for i, j in summary.items()))

comment_pdf(input_file="/content/final_test_2022.pdf"
            , list_filename_csv="/content/Sample Glossary for Daryl.csv"
            , output_file="/content/final_test_2022 comments.pdf"
            )